In [1]:
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from collections import defaultdict

In [2]:
# Read the data
with open('test.wp_target', 'r') as file:
    data = file.read()

print(f"Size of data:{len(data)}")
print(f"Data:{data[:100]}")

Size of data:50398417
Data:The wet marble floor pressed on his cheek like a thousand hands slapping his face frozen in time . S


In [3]:
data_list = data.split("\n")
print(f"size of data_list:{len(data_list)}")

size of data_list:15139


In [4]:
# Convert <newline> to \n
data_list = [x.replace("<newline>", "\n") for x in data_list]
# Append </BOT> and <EOT> to all the lines
data_list = ["</BOT>" + x + "<EOT>" for x in data_list]
print(data_list[0])

</BOT>The wet marble floor pressed on his cheek like a thousand hands slapping his face frozen in time . Smattering piss of rain ignored his indignant mumblings . His eyes fluttered . Pins and needs ran from finger to shoulder as he pushed back against the floor , contorting his aching body into a cross legged position . Last night was bad . He gathered that . His routine dullness of though crept inwards from the edges of his mind toward the black mist that veiled his most recent memories . He struggled to recall whatever he could n't recall but only for a moment before he decided it probably was n't worth the effort . 
 He glanced around the room for a few minutes before concluding that he probably did n't know where he was . His investigation was n't entirely fruitless , he discovered a mostly full bottle of vodka . It was cheap but would definitely get the job done . Taking a few swigs made it childishly easy to ignore that gigantic black cloud of fog blotting out whatever the hell 

In [5]:
# Split the data into word corpus for BPE procesing
data_corpus = [word for line in data_list for word in line.split()]
print(f"Size of data_corpus:{len(data_corpus)}")
print(f"Data_corpus:{data_corpus[:100]}")

Size of data_corpus:9777952
Data_corpus:['</BOT>The', 'wet', 'marble', 'floor', 'pressed', 'on', 'his', 'cheek', 'like', 'a', 'thousand', 'hands', 'slapping', 'his', 'face', 'frozen', 'in', 'time', '.', 'Smattering', 'piss', 'of', 'rain', 'ignored', 'his', 'indignant', 'mumblings', '.', 'His', 'eyes', 'fluttered', '.', 'Pins', 'and', 'needs', 'ran', 'from', 'finger', 'to', 'shoulder', 'as', 'he', 'pushed', 'back', 'against', 'the', 'floor', ',', 'contorting', 'his', 'aching', 'body', 'into', 'a', 'cross', 'legged', 'position', '.', 'Last', 'night', 'was', 'bad', '.', 'He', 'gathered', 'that', '.', 'His', 'routine', 'dullness', 'of', 'though', 'crept', 'inwards', 'from', 'the', 'edges', 'of', 'his', 'mind', 'toward', 'the', 'black', 'mist', 'that', 'veiled', 'his', 'most', 'recent', 'memories', '.', 'He', 'struggled', 'to', 'recall', 'whatever', 'he', 'could', "n't", 'recall']


In [6]:
# Initialize the vocabulary
special_tokens = {
    '</BOT>': 0,
    '</EOT>': 1
}
inv_special_tokens = { v:k for k,v in special_tokens.items() }
token_map = {}
token_map.update(special_tokens)
inv_token_map = {}
inv_token_map.update(inv_special_tokens)
bpe_codes = {}

In [7]:
# word splitting and counting
vocab = { '</BOT>' : 0, '</EOT>' : 1 }
for word in data_corpus:
    if word in special_tokens.keys():
        continue
    chars = list(word) + ['</w>']
    word_tuple = tuple(chars)
    vocab[word_tuple] = vocab.get(word_tuple, 0) + 1

In [8]:
# Collect unique symbols
symbols = set()
for word_tuple in vocab:
    symbols.update(word_tuple)

In [9]:
# Check the symbols
# There are many non-english symbols
print(symbols)

{'😂', '̝', '瑞', '́', 'w', '™', '̩', '🤔', 'آ', 'ん', 'ا', '生', 'M', 'ס', '김', 'È', '么', 'è', 'ḑ', 'ă', 'l', 'Ｘ', '|', 'ಠ', '𝘦', '͇', 'ň', '°', 'व', 'K', '5', 'ब', '″', 'R', '+', '»', 'æ', '̍', 'э', 'Ｙ', '§', 'j', '비', 'й', 'ゎ', 's', '은', 'ā', '…', '4', 'Վ', '2', 'भ', '\\', 'ß', 'ع', 'ج', '상', '̠', 'צ', 'H', 'ソ', 'ò', 'ԋ', '𝘨', 'ي', '͎', '~', 'ţ', 'W', '«', '‘', 'י', '̾', 'Ä', '★', 'ा', 'ט', '𝘑', 'â', '용', 'u', '−', '⌐', 'ժ', '€', '̨', '´', '͞', 'ء', '͏', 'ن', 'ַ', 'ú', 'g', '̴', '̪', 'Ѯ', '♬', '̋', 'ئ', '野', '$', 'ķ', '₰', 'ô', '͂', 'p', '}', 'ث', 'ه', 'ע', ')', '√', '̹', '⇔', 'Т', '̡', '니', '͛', 'ð', '💯', 'κ', 'Z', '͋', 'ơ', 'ָ', 'Q', 'ּ', '🔥', '☺', 'מ', 'ذ', '̟', 'E', 'þ', 'נ', 'س', '𝘸', 'ワ', '’', '̄', 'ư', 'р', '̓', 'ג', '⬆', 'य', 'ف', 'ｗ', 'i', '͌', 'े', 'आ', 'b', 'п', 'ש', '͖', 'ö', 'Ｍ', 'ｏ', '͢', 'ь', 'J', '̲', 'و', '̀', 'q', 'ε', '@', ',', 'е', '̑', 'ř', 'ļ', 'Ｅ', 'ツ', '𝘣', 'Á', '∴', '屎', 'Ð', '̥', 'Ｏ', '把', '\xad', 'إ', 'さ', '👌', '₴', '̣', '̆', '͡', '</w>', 'غ', 'd', '岁', '“', 'o

In [10]:
token_id = 2 # 0,1 are reserved for special tokens. 
print(type(token_map))
# If we wanted to do this dynamically we can do token_id = len(special_tokens)
for symbol in symbols:
    if symbol not in token_map:
        token_map[symbol] = token_id
        inv_token_map[token_id] = symbol
        token_id += 1

<class 'dict'>


In [11]:
inv_map = {v: k for k, v in token_map.items()}

In [12]:
print(token_map.keys())
print(inv_token_map.keys())

dict_keys(['</BOT>', '</EOT>', '😂', '̝', '瑞', '́', 'w', '™', '̩', '🤔', 'آ', 'ん', 'ا', '生', 'M', 'ס', '김', 'È', '么', 'è', 'ḑ', 'ă', 'l', 'Ｘ', '|', 'ಠ', '𝘦', '͇', 'ň', '°', 'व', 'K', '5', 'ब', '″', 'R', '+', '»', 'æ', '̍', 'э', 'Ｙ', '§', 'j', '비', 'й', 'ゎ', 's', '은', 'ā', '…', '4', 'Վ', '2', 'भ', '\\', 'ß', 'ع', 'ج', '상', '̠', 'צ', 'H', 'ソ', 'ò', 'ԋ', '𝘨', 'ي', '͎', '~', 'ţ', 'W', '«', '‘', 'י', '̾', 'Ä', '★', 'ा', 'ט', '𝘑', 'â', '용', 'u', '−', '⌐', 'ժ', '€', '̨', '´', '͞', 'ء', '͏', 'ن', 'ַ', 'ú', 'g', '̴', '̪', 'Ѯ', '♬', '̋', 'ئ', '野', '$', 'ķ', '₰', 'ô', '͂', 'p', '}', 'ث', 'ه', 'ע', ')', '√', '̹', '⇔', 'Т', '̡', '니', '͛', 'ð', '💯', 'κ', 'Z', '͋', 'ơ', 'ָ', 'Q', 'ּ', '🔥', '☺', 'מ', 'ذ', '̟', 'E', 'þ', 'נ', 'س', '𝘸', 'ワ', '’', '̄', 'ư', 'р', '̓', 'ג', '⬆', 'य', 'ف', 'ｗ', 'i', '͌', 'े', 'आ', 'b', 'п', 'ש', '͖', 'ö', 'Ｍ', 'ｏ', '͢', 'ь', 'J', '̲', 'و', '̀', 'q', 'ε', '@', ',', 'е', '̑', 'ř', 'ļ', 'Ｅ', 'ツ', '𝘣', 'Á', '∴', '屎', 'Ð', '̥', 'Ｏ', '把', '\xad', 'إ', 'さ', '👌', '₴', '̣', '̆', '͡', 

In [13]:
# get_pair_counts function
def get_pair_counts(vocab: dict[tuple[str], int]) -> dict[tuple[str, str], int]:
    """
    Count the frequency of each pair of symbols in the vocabulary
    by iterating over all words in the vocabulary and counting
    """
    pairs = {}
    for word, freq in vocab.items():
        symbols = word
        for i in range(len(symbols)-1): # There's no pair for last symbol
            pair = (symbols[i], symbols[i+1])
            pairs[pair] = pairs.get(pair, 0) + freq
    return pairs

In [ ]:
# merge_vocab function
from multiprocessing import Pool
def merge_vocab_primitive(pair: tuple[str, str], vocab:dict[tuple[str], int]) -> dict[tuple[str], int]:
    """
    Merge the most frequent pair in the vocabulary
    This is primitive version of merge_vocab using single thread
    to demonstrate algorithm
    """
    new_vocab = {}
    bigram = ''.join(pair) # join two symbols to create a bigram
    for word, freq in vocab.items():
        w = []
        i = 0
        while i < len(word)-1: # Reason for wile loop is inconsistent index combing
            # Merge pair if found
            if i < len(word) - 1 and word[i] == pair[0] and word[i+1] == pair[1]:
                w.append(bigram)
            else:
                w.append(word[i])
                i += 1
        new_vocab[tuple(w)] = freq
    return new_vocab

def merge_vocab(pair: tuple[str, str], vocab:dict[tuple[str], int]) -> dict[tuple[str], int]:
    """
    Merge the most frequent pair in the vocabulary
    This uses concurrent.futures.ThreadPoolExecutor to parallelize
    the merge_vocab_primitive function
    """
    bigram = ''.join(pair)

    def process_word(word_freq:tuple):
        word, freq = word_freq
        w = []
        i = 0
        while i < len(word)-1:
            if i < len(word) - 1 and word[i] == pair[0] and word[i+1] == pair[1]:
                w.append(bigram)
            else:
                w.append(word[i])
                i += 1
        return tuple(w), freq

    with Pool() as pool:
        new_vocab = dict(pool.map(process_word, vocab.items()))
    
    return new_vocab
    

In [ ]:
# # BPE Merges
# num_merges = 1
# bpe_codes = {}
# for i in tqdm(range(num_merges)):
#     pairs = get_pair_counts(vocab)
#     print(f"Best pair:{max(pairs, key=pairs.get)}")
#     print(f"BPE Codes:{bpe_codes}")
#     if not pairs:
#         break
#     best_pair = max(pairs, key=pairs.get)
#     vocab = merge_vocab(best_pair, vocab)
#     bpe_codes[best_pair] = i
#     new_symbol = ''.join(best_pair)
#     if new_symbol not in token_map:
#         token_map[new_symbol] = token_id
#         inv_token_map[token_id] = new_symbol
#         token_id += 1
    

  0%|          | 0/1 [00:00<?, ?it/s]

Best pair:('e', '</w>')
BPE Codes:{}
